# Prerequisite

In [1]:
import torch
import os
import numpy as np
import time
import glob
import shutil
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
import torchvision
import copy
import requests
import random

import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import models
from torchvision.datasets import CIFAR10, CIFAR100, MNIST, FashionMNIST, SVHN, ImageFolder
from torchvision import transforms
from torchvision.models import resnet
from torch.utils.data.dataset import Dataset 
from torch.utils.data import DataLoader
from PIL import Image
from glob import glob
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torchvision.utils import save_image
from torchvision.utils import make_grid

In [2]:
! pip install torchattacks
from torchattacks import FGSM, PGD

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
      Successfully uninstalled requests-2.28.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 10.0.1 which is incompatible.
be

In [3]:
!pip install gdown
import gdown
!pip install --upgrade --no-cache-dir gdown
!gdown 1xKuFbhpdLMBQ3pT2TmC9Pj9tKKIZgHaX

Downloading...
From: https://drive.google.com/uc?id=1xKuFbhpdLMBQ3pT2TmC9Pj9tKKIZgHaX
To: /kaggle/working/mnist_GLIDE_NormalClass0_6450.npy
100%|██████████████████████████████████████| 78.9M/78.9M [00:01<00:00, 57.8MB/s]


# Dataset

In [4]:
exposure_addr = './mnist_GLIDE_NormalClass0_6450.npy'

image_size = 32

train_attack_epses = [8/255]

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


In [6]:
class MyDataset_Binary(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, x, labels,transform):
        'Initialization'
        super(MyDataset_Binary, self).__init__()
        self.labels = labels
        self.x = x
        self.transform = transform

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.x)

  def __getitem__(self, index):
        'Generates one sample of data'
        if self.transform is None:
          x =  self.x[index]
          y = self.labels[index]
        else:
          x = self.transform(self.x[index])
          y = self.labels[index]
       
        return x, y
       


In [7]:
def show_samples(x):
  x = x.permute(0, 2, 3, 1).detach().cpu().numpy()
  img = image_grid(x)
  plt.figure(figsize=(8,8))
  plt.axis('off')
  plt.imshow(img)
  plt.show()
def image_grid(x):
  size =image_size# config.data.image_size train_model
  channels =3# config.data.num_channels
  img = x.reshape(-1, size, size, channels)
  w = int(np.sqrt(img.shape[0]))
  img = img.reshape((w, w, size, size, channels)).transpose((0, 2, 1, 3, 4)).reshape((w * size, w * size, channels))
  return img

In [9]:
orig_transform = transforms.Compose([
            transforms.Resize([image_size,image_size]),
            transforms.ToTensor()
        ])

trans_to_32 = transforms.Compose([  transforms.Resize((image_size,image_size))])

img_transform_32 = transforms.Compose([transforms.ToTensor(), transforms.Resize((image_size,image_size))])

In [10]:
def get_dataset(class_1):
  
  orig_transform = transforms.Compose([
            transforms.Resize([image_size,image_size]),
            transforms.ToTensor()
        ])

  trans_to_32 = transforms.Compose([  transforms.Resize((image_size,image_size))])

  img_transform_32_1 = transforms.Compose([transforms.Grayscale(3), transforms.ToTensor(), transforms.Resize((image_size,image_size))])

  img_transform_32_2 = transforms.Compose([transforms.Grayscale(3), transforms.ToTensor(), transforms.Resize((image_size,image_size))])

  train_1 = MNIST(f'./data/MNIST/train', train=True, download=True, transform=img_transform_32_1)
  images_train=[]
  for x,y in train_1:
      if y==class_1:
        images_train.append(x)
  train_tensor=torch.stack(images_train)
   
  testset = MNIST(f'./data/MNIST/test', train=False, download=True, transform=img_transform_32_1)

  test_labels=[]
  images_test=[]
  for x,y in testset:
    images_test.append(x)
    if y==class_1:
        test_labels.append(0)
    else:test_labels.append(1)
  images_test_tensor=torch.stack(images_test)

  mir_data_set_test = MyDataset_Binary(images_test_tensor, test_labels, trans_to_32)
  Test_loader_mir = DataLoader(mir_data_set_test, batch_size=10, shuffle=False)

  normal_train_count = train_tensor.shape[0] 

  exposure_data = torch.from_numpy(np.load(exposure_addr))
  exposure_data = (exposure_data + 1) / 2
   
  while len(train_tensor) > len(exposure_data):
    exposure_data = exposure_data.repeat(2, 1, 1, 1)
  
  exposure_data = exposure_data[:len(train_tensor)]
  
  fake_label=[1]*exposure_data.shape[0]
  fake_dataset = MyDataset_Binary(exposure_data,fake_label, trans_to_32)
  normal_label=[0]*train_tensor.shape[0]

  train_dataset = MyDataset_Binary(train_tensor, normal_label, trans_to_32)

  train_dev_sets = torch.utils.data.ConcatDataset([train_dataset, fake_dataset])
  bin_train_loader = torch.utils.data.DataLoader(train_dev_sets, shuffle=True, batch_size=10)
  
  print(train_tensor.shape, exposure_data.shape)

  return bin_train_loader, Test_loader_mir

In [12]:
_mean = (0.485, 0.456, 0.406)
_std = (0.229, 0.224, 0.225)

mu = torch.tensor(_mean).view(3,1,1).cuda()
std = torch.tensor(_std).view(3,1,1).cuda()

class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, stride:int=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out


class PreActBottleneck(nn.Module):
    '''Pre-activation version of the original Bottleneck module.'''
    expansion = 4

    def __init__(self, in_planes, planes, stride:int=1):
        super(PreActBottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out = self.conv3(F.relu(self.bn3(out)))
        out += shortcut
        return out


class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes:int=2):
        super(PreActResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.relu(self.bn(out))
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def PreActResNet18():
    return PreActResNet(PreActBlock, [2,2,2,2], num_classes=2)

def PreActResNet34():
    return PreActResNet(PreActBlock, [3,4,6,3])

def PreActResNet50():
    return PreActResNet(PreActBottleneck, [3,4,6,3])

def PreActResNet101():
    return PreActResNet(PreActBottleneck, [3,4,23,3])

def PreActResNet152():
    return PreActResNet(PreActBottleneck, [3,8,36,3])


# Model

In [13]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)

# Evaluation

In [11]:
def auc_softmax_adversarial(model, test_loader, test_attack):
  soft = torch.nn.Softmax(dim=1)
  anomaly_scores = []
  test_labels = []
  print('AUC Adversarial Softmax Started ...')
  with tqdm(test_loader, unit="batch") as tepoch:
 
    torch.cuda.empty_cache()
    for i, (data, target) in enumerate(tepoch):
      model.eval()
      data, target = data.to(device), target.to(device)
      labels = target.to(device)
      data = torch.clamp(data, 0, 1)
      adv_data = test_attack(data, target)
      output = model(adv_data)
      probs = soft(output)
      anomaly_scores += probs[:, 1].detach().cpu().numpy().tolist()
      test_labels += target.detach().cpu().numpy().tolist()

  auc = roc_auc_score(test_labels, anomaly_scores)
  print(f'AUC Adversairal - Softmax - score on epoch {epoch} is: {auc * 100}')
  return auc

from sklearn.metrics import roc_auc_score
def auc_softmax(model, test_loader   ):
  soft = torch.nn.Softmax(dim=1)
  anomaly_scores = []
  test_labels = []
  print('AUC Softmax Started ...')
  with torch.no_grad():
    with tqdm(test_loader, unit="batch") as tepoch:

      torch.cuda.empty_cache()
      for i, (data, target) in enumerate(tepoch):
        model.eval()
        data, target = data.to(device), target.to(device)
        data = torch.clamp(data, 0, 1)
        output = model(data)
        probs = soft(output)
        anomaly_scores += probs[:, 1].detach().cpu().numpy().tolist()
        test_labels += target.detach().cpu().numpy().tolist()

  auc = roc_auc_score(test_labels, anomaly_scores)
  print(f'AUC - Softmax - score on epoch {epoch} is: {auc * 100}')
  
  return auc


# Train

In [19]:
epochs = 4

In [20]:
for normal_class in range(0, 1):
    clean_auc = []
    adv_auc = []
    best_adv_auc = None

    bin_train_loader, Test_loader_mir = get_dataset(normal_class) 

    for p in range(len(train_attack_epses)):
        train_attack_eps = train_attack_epses[p]
        train_attack_steps = 10
        train_attack_alpha = 2.5 * train_attack_eps / train_attack_steps

        if image_size == 224: model = PretrainedModel(backbone).cuda()
        else: model = WideResNet(34, 2, 5, dropRate=0.0).cuda()
        
        train_attack = PGD(model, eps=train_attack_eps, alpha=train_attack_alpha, steps=train_attack_steps)
        optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00005)

        for epoch in range(epochs):
            total_loss, total_num = 0.0, 0
            loss = nn.CrossEntropyLoss()
            train_bar = tqdm(bin_train_loader, desc='Train Binary Classifier ...')
            for (img1, Y) in train_bar:
                model.train()

                data = torch.clamp(img1, 0, 1)
                adv_data = train_attack(data.cuda(), Y.cuda())

                optimizer.zero_grad()

                out_1 = model(adv_data.cuda()) 
                loss_ = loss(out_1,Y.cuda())  

                loss_.backward()
                optimizer.step()

                total_num += adv_data.size(0)
                total_loss += loss_.item() * adv_data.size(0)
                total_num += bin_train_loader.batch_size
                total_loss += loss_.item() * bin_train_loader.batch_size

                train_bar.set_description('Train Robust Epoch :  {} , Clf_B Robust Loss: {:.4f}'.format(epoch ,  total_loss / total_num))
        
        test_attack_steps = 100
        test_attack_alpha = 2.5 * (8/255) / test_attack_steps
        test_attack = PGD(model, eps=8/255, alpha=test_attack_alpha, steps=test_attack_steps)
        auc_softmax(model, Test_loader_mir)
        auc_softmax_adversarial(model, Test_loader_mir, test_attack)

torch.Size([5923, 3, 32, 32]) torch.Size([5923, 3, 32, 32])


Train Robust Epoch :  0 , Clf_B Robust Loss: 0.1672: 100%|██████████| 1185/1185 [01:38<00:00, 12.00it/s]
Train Robust Epoch :  1 , Clf_B Robust Loss: 0.0845: 100%|██████████| 1185/1185 [01:38<00:00, 12.02it/s]
Train Robust Epoch :  2 , Clf_B Robust Loss: 0.0620: 100%|██████████| 1185/1185 [01:38<00:00, 12.00it/s]
Train Robust Epoch :  3 , Clf_B Robust Loss: 0.0444: 100%|██████████| 1185/1185 [01:39<00:00, 11.96it/s]


AUC Softmax Started ...


100%|██████████| 1000/1000 [00:04<00:00, 225.07batch/s]


AUC - Softmax - score on epoch 3 is: 99.68031358885018
AUC Adversarial Softmax Started ...


  0%|          | 0/1000 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 1000/1000 [01:09<00:00, 14.29batch/s]

AUC Adversairal - Softmax - score on epoch 3 is: 97.91517715733742
